## TO DO
1) clean data  

    keep blanks for supplement video type - done
    latest book mark is anything but 'latest viewing' - done
    convert duration column - done
    
2) new columns  

    add, year, month, day of the week columns, hour column
    differentiate series vs movie using semi colon
    add colum for if there is a series which season
    
    

In [94]:
#packages
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

#some basic functions
def print_shape(dataframe,name):
    #prints dataframe shape and name, name must be in quotes
    # example print(dataframe,'my dataframe')
    print(name+" has: {} rows and {} columns".format(dataframe.shape[0],dataframe.shape[1]))
    

#some less basic functions
def movie_series_parser(x):
    #counts how many semicolons occour, if 1 or zero, assume a movie, if two or more assume a series
    #determines if text is a movie or series by looking to see if the word "Season" or "Part" after a semicolon
    #if it does, a new column will be created called "Season" in the dataframe and that text will be entered there
    #the rest of the function will then delete everthing after the word inclusive of ": Season" or ": Part"
    Movie_or_series = ""
    Season = ""
    Title_of_production = ""
    
    if x.count(":") <2:
        Movie_or_series = 'Movie'
        Title_of_production = x
        return Movie_or_series, Title_of_production, Season
    else:
        try:
            Season_search = re.search(r"(: Season \d)|(: Part \d)",x)
            Season = Season_search.group(1)  
        except:
            Season = 'None'   
        
        return Movie_or_series, Title_of_production, Season


In [95]:
#import data
viewing_activity_df = pd.read_csv('data/ViewingActivity.csv',parse_dates = ['Start Time'])
billing_history_df = pd.read_csv('data/BillingHistory.csv',parse_dates=['Transaction Date'])

#clean up viewing history data
#latest view is duplicate data and supplemental video are netflix ads
#convert duration to a floating numbers
viewing_activity_df = viewing_activity_df[viewing_activity_df['Latest Bookmark']!='Not latest view']
viewing_activity_df = viewing_activity_df[viewing_activity_df['Supplemental Video Type'].isnull()].reset_index(drop=True)
viewing_activity_df['Duration_floating_hours'] = pd.to_timedelta(viewing_activity_df['Duration'],unit='hours').dt.total_seconds()/3600
#add new columns, year, month, hour, weekday, add if movie or series, if series which season
viewing_activity_df['Year'] = viewing_activity_df['Start Time'].dt.year
viewing_activity_df['Month'] = viewing_activity_df['Start Time'].dt.month
viewing_activity_df['Hour'] = viewing_activity_df['Start Time'].dt.hour
viewing_activity_df['Week_day'] = viewing_activity_df['Start Time'].dt.dayofweek
weekday_dict = {0:'Sunday',1:'Monday',2:'Tuesday',3:'Wednesday',4:'Thursday',5:'Friday',6:'Saturday'}
viewing_activity_df["Week_day"].replace(weekday_dict, inplace=True)
#
viewing_activity_df['Movie_or_series'], viewing_activity_df['Title_of_production'], viewing_activity_df['Season'] = zip(*viewing_activity_df['Title'].apply(movie_series_parser))

print_shape(viewing_activity_df,'Viewing Activity')
viewing_activity_df.head()



AttributeError: 'NoneType' object has no attribute 'group'